In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
dir_var = 'pipeline_copy/perm/*.csv'
all_files = glob.glob(dir_var)
len(all_files)

2400

In [3]:
flist = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=False)  # <- HERE
    flist.append(df)
df_out = pd.concat(flist, axis=1, ignore_index=False)
df_out = df_out.drop('times', axis=1)
df_out['times'] = list(range(98))
df_out['times'] = df_out['times'].astype(int)
df_out.to_csv('pipeline_copy/perm_data_parallel_all_final_copy.csv')
df_out

/tmp/ipykernel_1168195/3909481839.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_out['times'] = list(range(98))


,b_scores_het_f100_maf0.4_cens0.8_perm6_cv1,b_scores_epi_f100_maf0.2_cens0.8_perm18_cv4,b_scores_epi_f100_maf0.2_cens0.4_perm9_cv0,b_scores_add_f100_maf0.4_cens0.1_perm3_cv1,b_scores_epi_f100_maf0.4_cens0.8_perm4_cv0,b_scores_add_f100_maf0.2_cens0.8_perm0_cv1,b_scores_het_f100_maf0.2_cens0.4_perm17_cv1,b_scores_epi_f100_maf0.4_cens0.1_perm13_cv1,b_scores_het_f100_maf0.2_cens0.1_perm16_cv4,b_scores_add_f100_maf0.4_cens0.8_perm3_cv2,...,b_scores_epi_f100_maf0.2_cens0.1_perm3_cv3,b_scores_me_f100_maf0.4_cens0.4_perm6_cv1,b_scores_epi_f100_maf0.2_cens0.4_perm4_cv2,b_scores_epi_f100_maf0.2_cens0.1_perm9_cv3,b_scores_het_f100_maf0.4_cens0.4_perm4_cv4,b_scores_het_f100_maf0.4_cens0.4_perm13_cv4,b_scores_me_f100_maf0.2_cens0.4_perm7_cv0,b_scores_het_f100_maf0.2_cens0.4_perm0_cv3,b_scores_het_f100_maf0.2_cens0.8_perm0_cv2,times
0,7.035603e-07,0.113169,0.297859,0.0,0.010661,0.0,0.036357,0.138683,0.011896,3.913026e-11,...,0.034581,0.038980,0.288812,0.034552,0.004991,0.004992,0.212444,0.060600,0.040047,0
1,4.214396e-06,0.144978,0.272287,0.0,0.010662,0.0,0.046764,0.203467,0.062684,3.528909e-10,...,0.034199,0.057172,0.292365,0.034163,0.004979,0.004978,0.203478,0.059326,0.110509,1
2,1.644435e-05,0.166144,0.258683,0.0,0.016090,0.0,0.051901,0.219342,0.139434,5.258260e-09,...,0.033978,0.079365,0.281580,0.033954,0.019789,0.019768,0.214447,0.063333,0.193859,2
3,5.345294e-03,0.186552,0.255718,0.0,0.016087,0.0,0.057083,0.237380,0.223165,2.898414e-08,...,0.033885,0.108900,0.280784,0.033881,0.024565,0.024527,0.234272,0.067650,0.298049,3
4,5.447186e-03,0.194346,0.259706,0.0,0.016086,0.0,0.057048,0.241878,0.249374,8.520617e-08,...,0.033862,0.128753,0.282924,0.033872,0.038959,0.038871,0.248535,0.076903,0.374042,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,NaN,NaN,NaN,NaN,0.870499,NaN,0.115731,0.027805,NaN,9.213737e-01,...,NaN,NaN,0.268670,NaN,0.089145,0.089159,NaN,0.082273,0.450828,93
94,NaN,NaN,NaN,NaN,0.870499,NaN,0.077333,0.027805,NaN,3.226783e-01,...,NaN,NaN,0.268670,NaN,0.089145,0.089159,NaN,0.082257,0.463513,94
95,NaN,NaN,NaN,NaN,0.773499,NaN,NaN,0.027805,NaN,NaN,...,NaN,NaN,0.268670,NaN,0.044712,0.044718,NaN,0.109113,0.441647,95
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027805,NaN,NaN,...,NaN,NaN,0.268670,NaN,NaN,NaN,NaN,0.110095,0.212014,96


In [4]:
def fun(df, col_name='b_scores_het_f100_maf0.4_cens0.8_perm6_cv1'):
    temp_df = df[[col_name, 'times']]
    temp_df = temp_df.dropna()
    try:
        val = np.trapz(temp_df[col_name], temp_df['times']) / (list(temp_df['times'])[-1] - list(temp_df['times'])[0])
    except Exception as e:
        display(temp_df)
        raise e
    return val

In [5]:
ibs_df = pd.DataFrame(columns=['Dataset', 'IBS'])
for col in df_out.columns:
    if 'times' not in col:
        val = fun(df_out, col)
        ibs_df = pd.concat([pd.DataFrame([[col,val]], columns=ibs_df.columns), ibs_df], ignore_index=True)
ibs_df = ibs_df.sort_values('Dataset').reset_index(drop=True)
ibs_df

,Dataset,IBS
0,b_scores_add_f100_maf0.2_cens0.1_perm0_cv0,0.061293
1,b_scores_add_f100_maf0.2_cens0.1_perm0_cv1,0.094936
2,b_scores_add_f100_maf0.2_cens0.1_perm0_cv2,0.061185
3,b_scores_add_f100_maf0.2_cens0.1_perm0_cv3,0.112867
4,b_scores_add_f100_maf0.2_cens0.1_perm0_cv4,0.049528
...,...,...
2395,b_scores_me_f100_maf0.4_cens0.8_perm9_cv0,0.420002
2396,b_scores_me_f100_maf0.4_cens0.8_perm9_cv1,0.280865
2397,b_scores_me_f100_maf0.4_cens0.8_perm9_cv2,0.286687
2398,b_scores_me_f100_maf0.4_cens0.8_perm9_cv3,0.289995


In [6]:
ibs_df[['b', 'scores', 'model', 'nfeat', 'maf', 'cens', 'perm', 'cv']] = pd.DataFrame(list(ibs_df['Dataset'].str.split('_')))
ibs_df.drop('Dataset', axis=1, inplace=True)
ibs_df.drop('scores', axis=1, inplace=True)
ibs_df.drop('b', axis=1, inplace=True)
ibs_df = ibs_df[['model', 'nfeat', 'maf', 'cens', 'perm', 'cv', 'IBS']]
ibs_df

,model,nfeat,maf,cens,perm,cv,IBS
0,add,f100,maf0.2,cens0.1,perm0,cv0,0.061293
1,add,f100,maf0.2,cens0.1,perm0,cv1,0.094936
2,add,f100,maf0.2,cens0.1,perm0,cv2,0.061185
3,add,f100,maf0.2,cens0.1,perm0,cv3,0.112867
4,add,f100,maf0.2,cens0.1,perm0,cv4,0.049528
...,...,...,...,...,...,...,...
2395,me,f100,maf0.4,cens0.8,perm9,cv0,0.420002
2396,me,f100,maf0.4,cens0.8,perm9,cv1,0.280865
2397,me,f100,maf0.4,cens0.8,perm9,cv2,0.286687
2398,me,f100,maf0.4,cens0.8,perm9,cv3,0.289995


In [7]:
# ibs_df_temp = ibs_df[(ibs_df['model'] == 'epi') & (ibs_df['maf']=='maf0.2') & (ibs_df['cens']=='cens0.1')]
# datapoints = ibs_df_temp['IBS'].ravel()
# pvalue = sum(datapoints > 0.1)/len(datapoints)
# pvalue

In [8]:
homedir = 'pipeline'
org_ibs = pd.read_csv(homedir + '/ibs_data_all.csv')

In [9]:
def calc_perm_pvalue(ibs_df, model, maf, cens):
    temp_df = ibs_df[(ibs_df['model'] == model) & (ibs_df['maf']==maf) & (ibs_df['cens']==cens)]
    data_points = temp_df['IBS'].ravel()
    model_label = model+'_f100_'+maf+'_'+cens
    org_val = fun(org_ibs[[model_label, 'times']], model_label)
    pvalue = sum(data_points < org_val)/len(data_points)
    print(model, maf, cens, org_val, pvalue)
    return pvalue

In [10]:
models = [ 'me', 'add', 'epi', 'het']
mafs = ['maf0.2', 'maf0.4']
censs = ['cens0.1', 'cens0.4', 'cens0.8']

In [11]:
pval_df = pd.DataFrame(columns=['Models', 'MAF', 'Cens', 'p-val'])
for model in models:
    for maf in mafs:
        for cens in censs:
            pval = calc_perm_pvalue(ibs_df, model, maf, cens)
            pval_df = pd.concat([pd.DataFrame([[model, maf, cens, pval]], columns=pval_df.columns), pval_df], ignore_index=True)
pval_df = pval_df.reindex(index=pval_df.index[::-1]).reset_index(drop=True)
pval_df

me maf0.2 cens0.1 0.15869061399779694 0.0
me maf0.2 cens0.4 0.1797141484547237 0.2
me maf0.2 cens0.8 0.38818443856136003 0.6
me maf0.4 cens0.1 0.15726819946249204 0.0
me maf0.4 cens0.4 0.17812201281772594 0.0
me maf0.4 cens0.8 0.24070032917121242 0.04
add maf0.2 cens0.1 0.055960214778596584 0.2
add maf0.2 cens0.4 0.17521861086231227 0.6
add maf0.2 cens0.8 0.12323417324591347 0.04
add maf0.4 cens0.1 0.15226727233290505 0.26
add maf0.4 cens0.4 0.12061779243634398 0.2
add maf0.4 cens0.8 0.21981894929895907 0.28
epi maf0.2 cens0.1 0.07328174611756348 0.2
epi maf0.2 cens0.4 0.20109230134382672 0.2
epi maf0.2 cens0.8 0.3084432946807199 0.57
epi maf0.4 cens0.1 0.15533445677433105 0.02
epi maf0.4 cens0.4 0.2831109064800164 0.43
epi maf0.4 cens0.8 0.17654438232452965 0.8
het maf0.2 cens0.1 0.14149190704958192 0.08
het maf0.2 cens0.4 0.18536482722830708 0.0
het maf0.2 cens0.8 0.3685898357993639 0.25
het maf0.4 cens0.1 0.09294378887355878 0.38
het maf0.4 cens0.4 0.13660758533844794 0.24
het maf0.

,Models,MAF,Cens,p-val
0,me,maf0.2,cens0.1,0.00
1,me,maf0.2,cens0.4,0.20
2,me,maf0.2,cens0.8,0.60
3,me,maf0.4,cens0.1,0.00
4,me,maf0.4,cens0.4,0.00
5,me,maf0.4,cens0.8,0.04
6,add,maf0.2,cens0.1,0.20
7,add,maf0.2,cens0.4,0.60
8,add,maf0.2,cens0.8,0.04
9,add,maf0.4,cens0.1,0.26


In [14]:
pval_df[pval_df['p-val'] <= 0.05]

,Models,MAF,Cens,p-val
0,me,maf0.2,cens0.1,0.00
3,me,maf0.4,cens0.1,0.00
4,me,maf0.4,cens0.4,0.00
5,me,maf0.4,cens0.8,0.04
8,add,maf0.2,cens0.8,0.04
15,epi,maf0.4,cens0.1,0.02
19,het,maf0.2,cens0.4,0.00
